# Using Callbacks in Keras

In this notebook, we well see how to use pre-defined and custom callbacks in Keras for tasks such as chekpointing, learning rate scheduling, etc.

We'll use the same simple dataset and linear model of the previous notebook.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

2021-12-22 12:16:08.763485: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-22 12:16:08.763507: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


##### Download the Auto-MPG dataset
 
Let's use the Auto-MPG dataset (seen in a previous notebook).

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

dataset = pd.read_csv(url, names=column_names, na_values='?', comment='\t', sep=' ', skipinitialspace=True)
dataset = dataset.dropna()
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
393,27.0,4,140.0,86.0,2790.0,15.6,82,0,0,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,1,0,0
395,32.0,4,135.0,84.0,2295.0,11.6,82,0,0,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,0,0,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,0,0,1


In [3]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

##### Build the model

Let's build a simple linear regression model (seen in a previous notebook) to test different callbacks during its training.

We use a `get_model()` function so that we can re-create the model from scratch multiple times with a single instruction.

In [4]:
def get_model():
    normalizer = preprocessing.Normalization(input_shape=[9,])
    normalizer.adapt(np.array(train_features))
    
    model = keras.Sequential([
        normalizer,
        layers.Dense(units=1)
    ])
    
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.1),
        loss='mse', metrics=['mae', 'mse']
    )
    
    return model

## Early Stopping callback

Let's use the early stopping callback to stop training when it reaches stability.

The `monitor` parameter specifies the loss/metric to be monitored, and the `patience` parameters specifies the number of non-improving epochs to wait before stopping. 

In [5]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1)

# re-create the model to restart training every time
model = get_model()
history = model.fit(train_features, train_labels, epochs=200, validation_split = 0.2, callbacks=[es_callback])

2021-12-22 12:16:12.406346: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-22 12:16:12.406370: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-22 12:16:12.406388: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (matteo-Inspiron-7591-2n1): /proc/driver/nvidia/version does not exist
2021-12-22 12:16:12.406572: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
8/8 [==============================] - 0s 18ms/step - loss: 585.2602 - mae: 22.9254 - mse: 585.2602 - val_loss: 559.5980 - val_mae: 22.9405 - val_mse: 559.5980
Epoch 2/200
8/8 [==============================] - 0s 3ms/step - loss: 517.0109 - mae: 22.1718 - mse: 517.0109 - val_loss: 499.8539 - val_mae: 22.0346 - val_mse: 499.8539
Epoch 3/200
8/8 [==============================] - 0s 4ms/step - loss: 471.5683 - mae: 21.3457 - mse: 471.5683 - val_loss: 464.2890 - val_mae: 21.2402 - val_mse: 464.2890
Epoch 4/200
8/8 [==============================] - 0s 4ms/step - loss: 436.6931 - mae: 20.5261 - mse: 436.6931 - val_loss: 433.9763 - val_mae: 20.5267 - val_mse: 433.9763
Epoch 5/200
8/8 [==============================] - 0s 3ms/step - loss: 404.5422 - mae: 19.7431 - mse: 404.5422 - val_loss: 405.7402 - val_mae: 19.8354 - val_mse: 405.7402
Epoch 6/200
8/8 [==============================] - 0s 4ms/step - loss: 374.8262 - mae: 18.9781 - mse: 374.8262 - val_loss: 378.3346 - val_mae: 1

Epoch 50/200
8/8 [==============================] - 0s 4ms/step - loss: 13.5871 - mae: 2.7732 - mse: 13.5871 - val_loss: 13.8457 - val_mae: 2.7878 - val_mse: 13.8457
Epoch 51/200
8/8 [==============================] - 0s 3ms/step - loss: 13.2251 - mae: 2.7333 - mse: 13.2251 - val_loss: 13.3674 - val_mae: 2.7449 - val_mse: 13.3674
Epoch 52/200
8/8 [==============================] - 0s 3ms/step - loss: 13.0073 - mae: 2.7142 - mse: 13.0073 - val_loss: 13.1358 - val_mae: 2.7118 - val_mse: 13.1358
Epoch 53/200
8/8 [==============================] - 0s 3ms/step - loss: 12.8484 - mae: 2.6986 - mse: 12.8484 - val_loss: 12.7723 - val_mae: 2.6833 - val_mse: 12.7723
Epoch 54/200
8/8 [==============================] - 0s 3ms/step - loss: 12.6430 - mae: 2.6695 - mse: 12.6430 - val_loss: 12.4197 - val_mae: 2.6655 - val_mse: 12.4197
Epoch 55/200
8/8 [==============================] - 0s 4ms/step - loss: 12.4618 - mae: 2.6381 - mse: 12.4618 - val_loss: 12.2297 - val_mae: 2.6457 - val_mse: 12.2297
Epoc

As you can see, the training stopped after about 60/70 epochs, rather than running for the entire 200 epochs specified in `fit()`.

## Checkpoint Callback

Let's add a second callback to save a model checkpoint after every epoch. Notice that we can pass multiple callbacks at the same time to `fit()`.

In [6]:
cp_callback = keras.callbacks.ModelCheckpoint(
    './callback_test_chkp/chkp_{epoch:02d}',
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=False,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

In [7]:
model = get_model()
history = model.fit(train_features, train_labels, epochs=200, validation_split = 0.2,
                                callbacks=[es_callback, cp_callback])

Epoch 1/200
1/8 [==>...........................] - ETA: 1s - loss: 612.8845 - mae: 23.1756 - mse: 612.8845

2021-12-22 12:16:15.643760: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./callback_test_chkp/chkp_01/assets
8/8 [==============================] - 1s 68ms/step - loss: 579.1166 - mae: 22.9874 - mse: 579.1166 - val_loss: 546.4377 - val_mae: 22.8021 - val_mse: 546.4377
Epoch 2/200
8/8 [==============================] - 0s 45ms/step - loss: 515.4190 - mae: 22.2004 - mse: 515.4190 - val_loss: 497.4308 - val_mae: 21.9939 - val_mse: 497.4308
Epoch 3/200
8/8 [==============================] - 0s 59ms/step - loss: 469.2399 - mae: 21.2954 - mse: 469.2399 - val_loss: 463.3602 - val_mae: 21.2483 - val_mse: 463.3602
Epoch 4/200
8/8 [==============================] - 0s 45ms/step - loss: 435.2071 - mae: 20.5104 - mse: 435.2071 - val_loss: 433.5208 - val_mae: 20.5422 - val_mse: 433.5208
Epoch 5/200
8/8 [==============================] - 0s 45ms/step - loss: 403.2576 - mae: 19.7287 - mse: 403.2576 - val_loss: 405.0819 - val_mae: 19.8416 - val_mse: 405.0819
Epoch 6/200
8/8 [==============================] - 0s 46ms/step - loss: 373.3617 

Epoch 26/200
8/8 [==============================] - 1s 75ms/step - loss: 64.9741 - mae: 7.2943 - mse: 64.9741 - val_loss: 70.8682 - val_mae: 7.7087 - val_mse: 70.8682
Epoch 27/200
8/8 [==============================] - 0s 46ms/step - loss: 58.9196 - mae: 6.8943 - mse: 58.9196 - val_loss: 64.6649 - val_mae: 7.3153 - val_mse: 64.6649
Epoch 28/200
8/8 [==============================] - 0s 45ms/step - loss: 53.7143 - mae: 6.5192 - mse: 53.7143 - val_loss: 59.5887 - val_mae: 6.9589 - val_mse: 59.5887
Epoch 29/200
8/8 [==============================] - 0s 40ms/step - loss: 48.9450 - mae: 6.1474 - mse: 48.9450 - val_loss: 54.4971 - val_mae: 6.5941 - val_mse: 54.4971
Epoch 30/200
8/8 [==============================] - 0s 48ms/step - loss: 44.5900 - mae: 5.8091 - mse: 44.5900 - val_loss: 49.5642 - val_mae: 6.2400 - val_mse: 49.5642
Epoch 31/200
8/8 [==============================] - 0s 44ms/step - loss: 40.6464 - mae: 5.4994 - mse: 40.6464 - val_loss: 45.3325 - val_mae: 5.9104 - val_mse: 45.332

Epoch 51/200
8/8 [==============================] - 0s 46ms/step - loss: 13.1645 - mae: 2.7237 - mse: 13.1645 - val_loss: 13.4079 - val_mae: 2.7518 - val_mse: 13.4079
Epoch 52/200
8/8 [==============================] - 0s 48ms/step - loss: 12.9103 - mae: 2.6950 - mse: 12.9103 - val_loss: 13.0089 - val_mae: 2.7114 - val_mse: 13.0089
Epoch 53/200
8/8 [==============================] - 0s 68ms/step - loss: 12.6711 - mae: 2.6747 - mse: 12.6711 - val_loss: 12.6923 - val_mae: 2.6858 - val_mse: 12.6923
Epoch 54/200
8/8 [==============================] - 0s 48ms/step - loss: 12.5284 - mae: 2.6528 - mse: 12.5284 - val_loss: 12.4578 - val_mae: 2.6651 - val_mse: 12.4578
Epoch 55/200
8/8 [==============================] - 0s 47ms/step - loss: 12.3350 - mae: 2.6266 - mse: 12.3350 - val_loss: 12.1914 - val_mae: 2.6423 - val_mse: 12.1914
Epoch 56/200
8/8 [==============================] - 0s 51ms/step - loss: 12.2769 - mae: 2.6307 - mse: 12.2769 - val_loss: 11.8556 - val_mae: 2.6098 - val_mse: 11.855

Epoch 00075: early stopping


##### Restore a saved checkpoint

Let's try loading back two different models, and let's evaluate them on training data.

In [8]:
model_epoch1 = keras.models.load_model('./callback_test_chkp/chkp_01')
model_epoch1.evaluate(train_features, train_labels,)

10/10 [==============================] - 0s 889us/step - loss: 534.7635 - mae: 22.5113 - mse: 534.7635


[534.7635498046875, 22.51128578186035, 534.7635498046875]

In [9]:
model_epoch10 = keras.models.load_model('./callback_test_chkp/chkp_10')
model_epoch10.evaluate(train_features, train_labels,)

10/10 [==============================] - 0s 1ms/step - loss: 263.2537 - mae: 15.8052 - mse: 263.2537


[263.253662109375, 15.80517578125, 263.253662109375]

## Learning Rate Scheduling

Let's try to change the learning rate by reducing it by 0.01 after every epoch. This is just to demonstrate LR scheduling, it is not a particularly useful scheduling mechanism.

In [10]:
def my_schedule(epoch, lr):
    return max(lr - 0.01, 0.01)

Test if the schedule works for different input LR values.

In [11]:
print(my_schedule(1, 0.05))
print(my_schedule(1, 0.01))

0.04
0.01


In [12]:
lr_callback = keras.callbacks.LearningRateScheduler(my_schedule, verbose=1)

In [13]:
model = get_model()
history = model.fit(train_features, train_labels, epochs=200, validation_split = 0.2,
                                callbacks=[lr_callback, es_callback])


Epoch 00001: LearningRateScheduler setting learning rate to 0.09000000149011612.
Epoch 1/200
8/8 [==============================] - 1s 25ms/step - loss: 568.0912 - mae: 22.9338 - mse: 568.0912 - val_loss: 561.9692 - val_mae: 23.1512 - val_mse: 561.9692 - lr: 0.0900

Epoch 00002: LearningRateScheduler setting learning rate to 0.08000000357627869.
Epoch 2/200
8/8 [==============================] - 0s 4ms/step - loss: 514.5397 - mae: 22.1865 - mse: 514.5397 - val_loss: 522.3202 - val_mae: 22.4901 - val_mse: 522.3202 - lr: 0.0800

Epoch 00003: LearningRateScheduler setting learning rate to 0.07000000566244126.
Epoch 3/200
8/8 [==============================] - 0s 4ms/step - loss: 483.5588 - mae: 21.5829 - mse: 483.5588 - val_loss: 494.6035 - val_mae: 21.9290 - val_mse: 494.6035 - lr: 0.0700

Epoch 00004: LearningRateScheduler setting learning rate to 0.06000000774860382.
Epoch 4/200
8/8 [==============================] - 0s 7ms/step - loss: 460.8472 - mae: 21.0891 - mse: 460.8472 - val_lo


Epoch 00033: LearningRateScheduler setting learning rate to 0.01.
Epoch 33/200
8/8 [==============================] - 0s 4ms/step - loss: 333.0726 - mae: 17.8545 - mse: 333.0726 - val_loss: 360.4099 - val_mae: 18.6368 - val_mse: 360.4099 - lr: 0.0100

Epoch 00034: LearningRateScheduler setting learning rate to 0.01.
Epoch 34/200
8/8 [==============================] - 0s 4ms/step - loss: 330.3840 - mae: 17.7800 - mse: 330.3840 - val_loss: 357.6510 - val_mae: 18.5631 - val_mse: 357.6510 - lr: 0.0100

Epoch 00035: LearningRateScheduler setting learning rate to 0.01.
Epoch 35/200
8/8 [==============================] - 0s 3ms/step - loss: 327.8230 - mae: 17.7091 - mse: 327.8230 - val_loss: 354.8318 - val_mae: 18.4883 - val_mse: 354.8318 - lr: 0.0100

Epoch 00036: LearningRateScheduler setting learning rate to 0.01.
Epoch 36/200
8/8 [==============================] - 0s 4ms/step - loss: 325.0799 - mae: 17.6323 - mse: 325.0799 - val_loss: 352.2760 - val_mae: 18.4178 - val_mse: 352.2760 - lr:

8/8 [==============================] - 0s 4ms/step - loss: 255.1996 - mae: 15.5389 - mse: 255.1996 - val_loss: 280.3805 - val_mae: 16.3506 - val_mse: 280.3805 - lr: 0.0100

Epoch 00066: LearningRateScheduler setting learning rate to 0.01.
Epoch 66/200
8/8 [==============================] - 0s 4ms/step - loss: 253.1167 - mae: 15.4714 - mse: 253.1167 - val_loss: 278.2611 - val_mae: 16.2838 - val_mse: 278.2611 - lr: 0.0100

Epoch 00067: LearningRateScheduler setting learning rate to 0.01.
Epoch 67/200
8/8 [==============================] - 0s 4ms/step - loss: 250.8825 - mae: 15.3993 - mse: 250.8825 - val_loss: 276.0200 - val_mae: 16.2145 - val_mse: 276.0200 - lr: 0.0100

Epoch 00068: LearningRateScheduler setting learning rate to 0.01.
Epoch 68/200
8/8 [==============================] - 0s 4ms/step - loss: 248.7883 - mae: 15.3314 - mse: 248.7883 - val_loss: 273.7058 - val_mae: 16.1442 - val_mse: 273.7058 - lr: 0.0100

Epoch 00069: LearningRateScheduler setting learning rate to 0.01.
Epoch


Epoch 00098: LearningRateScheduler setting learning rate to 0.01.
Epoch 98/200
8/8 [==============================] - 0s 3ms/step - loss: 190.6013 - mae: 13.3139 - mse: 190.6013 - val_loss: 212.3131 - val_mae: 14.1230 - val_mse: 212.3131 - lr: 0.0100

Epoch 00099: LearningRateScheduler setting learning rate to 0.01.
Epoch 99/200
8/8 [==============================] - 0s 4ms/step - loss: 188.8649 - mae: 13.2484 - mse: 188.8649 - val_loss: 210.5586 - val_mae: 14.0597 - val_mse: 210.5586 - lr: 0.0100

Epoch 00100: LearningRateScheduler setting learning rate to 0.01.
Epoch 100/200
8/8 [==============================] - 0s 3ms/step - loss: 187.1521 - mae: 13.1832 - mse: 187.1521 - val_loss: 208.6789 - val_mae: 13.9933 - val_mse: 208.6789 - lr: 0.0100

Epoch 00101: LearningRateScheduler setting learning rate to 0.01.
Epoch 101/200
8/8 [==============================] - 0s 3ms/step - loss: 185.4381 - mae: 13.1184 - mse: 185.4381 - val_loss: 206.9275 - val_mae: 13.9302 - val_mse: 206.9275 - l

8/8 [==============================] - 0s 4ms/step - loss: 141.0119 - mae: 11.3154 - mse: 141.0119 - val_loss: 159.2273 - val_mae: 12.1090 - val_mse: 159.2273 - lr: 0.0100

Epoch 00131: LearningRateScheduler setting learning rate to 0.01.
Epoch 131/200
8/8 [==============================] - 0s 4ms/step - loss: 139.6711 - mae: 11.2559 - mse: 139.6711 - val_loss: 157.7331 - val_mae: 12.0477 - val_mse: 157.7331 - lr: 0.0100

Epoch 00132: LearningRateScheduler setting learning rate to 0.01.
Epoch 132/200
8/8 [==============================] - 0s 4ms/step - loss: 138.4000 - mae: 11.2006 - mse: 138.4000 - val_loss: 156.2587 - val_mae: 11.9875 - val_mse: 156.2587 - lr: 0.0100

Epoch 00133: LearningRateScheduler setting learning rate to 0.01.
Epoch 133/200
8/8 [==============================] - 0s 3ms/step - loss: 136.9733 - mae: 11.1375 - mse: 136.9733 - val_loss: 154.7830 - val_mae: 11.9262 - val_mse: 154.7830 - lr: 0.0100

Epoch 00134: LearningRateScheduler setting learning rate to 0.01.
Ep


Epoch 00163: LearningRateScheduler setting learning rate to 0.01.
Epoch 163/200
8/8 [==============================] - 0s 4ms/step - loss: 101.4803 - mae: 9.4337 - mse: 101.4803 - val_loss: 116.1239 - val_mae: 10.1902 - val_mse: 116.1239 - lr: 0.0100

Epoch 00164: LearningRateScheduler setting learning rate to 0.01.
Epoch 164/200
8/8 [==============================] - 0s 4ms/step - loss: 100.4178 - mae: 9.3782 - mse: 100.4178 - val_loss: 114.9232 - val_mae: 10.1334 - val_mse: 114.9232 - lr: 0.0100

Epoch 00165: LearningRateScheduler setting learning rate to 0.01.
Epoch 165/200
8/8 [==============================] - 0s 4ms/step - loss: 99.3914 - mae: 9.3244 - mse: 99.3914 - val_loss: 113.7689 - val_mae: 10.0775 - val_mse: 113.7689 - lr: 0.0100

Epoch 00166: LearningRateScheduler setting learning rate to 0.01.
Epoch 166/200
8/8 [==============================] - 0s 4ms/step - loss: 98.3831 - mae: 9.2714 - mse: 98.3831 - val_loss: 112.5898 - val_mae: 10.0208 - val_mse: 112.5898 - lr: 0.0

8/8 [==============================] - 0s 4ms/step - loss: 71.7445 - mae: 7.7358 - mse: 71.7445 - val_loss: 83.1704 - val_mae: 8.4415 - val_mse: 83.1704 - lr: 0.0100

Epoch 00197: LearningRateScheduler setting learning rate to 0.01.
Epoch 197/200
8/8 [==============================] - 0s 4ms/step - loss: 70.9843 - mae: 7.6883 - mse: 70.9843 - val_loss: 82.3652 - val_mae: 8.3930 - val_mse: 82.3652 - lr: 0.0100

Epoch 00198: LearningRateScheduler setting learning rate to 0.01.
Epoch 198/200
8/8 [==============================] - 0s 6ms/step - loss: 70.2509 - mae: 7.6414 - mse: 70.2509 - val_loss: 81.4628 - val_mae: 8.3413 - val_mse: 81.4628 - lr: 0.0100

Epoch 00199: LearningRateScheduler setting learning rate to 0.01.
Epoch 199/200
8/8 [==============================] - 0s 4ms/step - loss: 69.4702 - mae: 7.5929 - mse: 69.4702 - val_loss: 80.6324 - val_mae: 8.2918 - val_mse: 80.6324 - lr: 0.0100

Epoch 00200: LearningRateScheduler setting learning rate to 0.01.
Epoch 200/200
8/8 [=======

## Custom Callback N.1

Let's write a simple custom callback that logs the loss and metrics values after every batch, epoch, etc.

In [14]:
class CustomLogger(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        print("Starting training; log content: {}".format(logs))

    def on_train_end(self, logs=None):
        print("Stop training; got log content: {}".format(logs))

    def on_epoch_end(self, epoch, logs=None):
        print("End epoch {} of training; log content: {}".format(epoch, logs))

    def on_train_batch_end(self, batch, logs=None):
        print("...Training: end of batch {}; log content: {}".format(batch, logs))


In [15]:
log_callback = CustomLogger()

model = get_model()
history = model.fit(train_features, train_labels, epochs=200,
                                verbose=0, #verbose=0 to avoid mixing our prints and those of standard keras
                                validation_split = 0.2,
                                callbacks=[log_callback]
)

Starting training; log content: {}
...Training: end of batch 0; log content: {'loss': 640.1168212890625, 'mae': 23.924179077148438, 'mse': 640.1168212890625}
...Training: end of batch 1; log content: {'loss': 601.4976196289062, 'mae': 23.367958068847656, 'mse': 601.4976196289062}
...Training: end of batch 2; log content: {'loss': 603.247802734375, 'mae': 23.63275718688965, 'mse': 603.247802734375}
...Training: end of batch 3; log content: {'loss': 586.8490600585938, 'mae': 23.314189910888672, 'mse': 586.8490600585938}
...Training: end of batch 4; log content: {'loss': 568.4512939453125, 'mae': 22.980281829833984, 'mse': 568.4512939453125}
...Training: end of batch 5; log content: {'loss': 558.9595336914062, 'mae': 22.837554931640625, 'mse': 558.9595336914062}
...Training: end of batch 6; log content: {'loss': 556.0956420898438, 'mae': 22.81576919555664, 'mse': 556.0956420898438}
...Training: end of batch 7; log content: {'loss': 561.2073974609375, 'mae': 22.94468879699707, 'mse': 561.2

End epoch 10 of training; log content: {'loss': 250.2487030029297, 'mae': 15.372559547424316, 'mse': 250.2487030029297, 'val_loss': 259.7608642578125, 'val_mae': 15.729279518127441, 'val_mse': 259.7608642578125}
...Training: end of batch 0; log content: {'loss': 219.69247436523438, 'mae': 14.310229301452637, 'mse': 219.69247436523438}
...Training: end of batch 1; log content: {'loss': 249.23289489746094, 'mae': 15.20467758178711, 'mse': 249.23289489746094}
...Training: end of batch 2; log content: {'loss': 241.60801696777344, 'mae': 14.985664367675781, 'mse': 241.60801696777344}
...Training: end of batch 3; log content: {'loss': 240.0593719482422, 'mae': 14.980049133300781, 'mse': 240.0593719482422}
...Training: end of batch 4; log content: {'loss': 239.7289581298828, 'mae': 14.99125862121582, 'mse': 239.7289581298828}
...Training: end of batch 5; log content: {'loss': 230.0460662841797, 'mae': 14.690673828125, 'mse': 230.0460662841797}
...Training: end of batch 6; log content: {'loss'

End epoch 18 of training; log content: {'loss': 124.58759307861328, 'mae': 10.566314697265625, 'mse': 124.58759307861328, 'val_loss': 131.84349060058594, 'val_mae': 10.96011734008789, 'val_mse': 131.84349060058594}
...Training: end of batch 0; log content: {'loss': 116.57064819335938, 'mae': 10.363588333129883, 'mse': 116.57064819335938}
...Training: end of batch 1; log content: {'loss': 119.81245422363281, 'mae': 10.290020942687988, 'mse': 119.81245422363281}
...Training: end of batch 2; log content: {'loss': 112.76578521728516, 'mae': 10.046494483947754, 'mse': 112.76578521728516}
...Training: end of batch 3; log content: {'loss': 115.21142578125, 'mae': 10.063802719116211, 'mse': 115.21142578125}
...Training: end of batch 4; log content: {'loss': 114.1043930053711, 'mae': 10.024415969848633, 'mse': 114.1043930053711}
...Training: end of batch 5; log content: {'loss': 113.20852661132812, 'mae': 9.982121467590332, 'mse': 113.20852661132812}
...Training: end of batch 6; log content: {'

...Training: end of batch 5; log content: {'loss': 54.75857925415039, 'mae': 6.580890655517578, 'mse': 54.75857925415039}
...Training: end of batch 6; log content: {'loss': 54.31869888305664, 'mae': 6.525269985198975, 'mse': 54.31869888305664}
...Training: end of batch 7; log content: {'loss': 53.55525207519531, 'mae': 6.506133556365967, 'mse': 53.55525207519531}
End epoch 27 of training; log content: {'loss': 53.55525207519531, 'mae': 6.506133556365967, 'mse': 53.55525207519531, 'val_loss': 58.736698150634766, 'val_mae': 6.916505813598633, 'val_mse': 58.736698150634766}
...Training: end of batch 0; log content: {'loss': 61.73976135253906, 'mae': 6.933187484741211, 'mse': 61.73976135253906}
...Training: end of batch 1; log content: {'loss': 47.07685089111328, 'mae': 6.088812828063965, 'mse': 47.07685089111328}
...Training: end of batch 2; log content: {'loss': 48.54850387573242, 'mae': 6.21239709854126, 'mse': 48.54850387573242}
...Training: end of batch 3; log content: {'loss': 47.838

End epoch 34 of training; log content: {'loss': 29.056724548339844, 'mae': 4.451137065887451, 'mse': 29.056724548339844, 'val_loss': 32.411991119384766, 'val_mae': 4.766209125518799, 'val_mse': 32.411991119384766}
...Training: end of batch 0; log content: {'loss': 21.57598114013672, 'mae': 3.6450138092041016, 'mse': 21.57598114013672}
...Training: end of batch 1; log content: {'loss': 19.100364685058594, 'mae': 3.505295753479004, 'mse': 19.100364685058594}
...Training: end of batch 2; log content: {'loss': 25.434341430664062, 'mae': 3.9524612426757812, 'mse': 25.434341430664062}
...Training: end of batch 3; log content: {'loss': 27.3979549407959, 'mae': 4.21675968170166, 'mse': 27.3979549407959}
...Training: end of batch 4; log content: {'loss': 26.446508407592773, 'mae': 4.189908027648926, 'mse': 26.446508407592773}
...Training: end of batch 5; log content: {'loss': 27.263036727905273, 'mae': 4.226528167724609, 'mse': 27.263036727905273}
...Training: end of batch 6; log content: {'los

...

...Training: end of batch 6; log content: {'loss': 11.480706214904785, 'mae': 2.591892719268799, 'mse': 11.480706214904785}
...Training: end of batch 7; log content: {'loss': 11.413748741149902, 'mae': 2.599073886871338, 'mse': 11.413748741149902}
End epoch 155 of training; log content: {'loss': 11.413748741149902, 'mae': 2.599073886871338, 'mse': 11.413748741149902, 'val_loss': 10.048480033874512, 'val_mae': 2.421628713607788, 'val_mse': 10.048480033874512}
...Training: end of batch 0; log content: {'loss': 8.000626564025879, 'mae': 2.3386712074279785, 'mse': 8.000626564025879}
...Training: end of batch 1; log content: {'loss': 11.644508361816406, 'mae': 2.4196040630340576, 'mse': 11.644508361816406}
...Training: end of batch 2; log content: {'loss': 11.344955444335938, 'mae': 2.4757702350616455, 'mse': 11.344955444335938}
...Training: end of batch 3; log content: {'loss': 11.897747039794922, 'mae': 2.5746986865997314, 'mse': 11.897747039794922}
...Training: end of batch 4; log conten

End epoch 164 of training; log content: {'loss': 11.432588577270508, 'mae': 2.606843948364258, 'mse': 11.432588577270508, 'val_loss': 9.924283027648926, 'val_mae': 2.4150688648223877, 'val_mse': 9.924283027648926}
...Training: end of batch 0; log content: {'loss': 10.877325057983398, 'mae': 2.730469226837158, 'mse': 10.877325057983398}
...Training: end of batch 1; log content: {'loss': 10.55762767791748, 'mae': 2.592839479446411, 'mse': 10.55762767791748}
...Training: end of batch 2; log content: {'loss': 9.60256576538086, 'mae': 2.506728410720825, 'mse': 9.60256576538086}
...Training: end of batch 3; log content: {'loss': 8.848785400390625, 'mae': 2.3971240520477295, 'mse': 8.848785400390625}
...Training: end of batch 4; log content: {'loss': 9.819811820983887, 'mae': 2.495211601257324, 'mse': 9.819811820983887}
...Training: end of batch 5; log content: {'loss': 11.377223014831543, 'mae': 2.5955944061279297, 'mse': 11.377223014831543}
...Training: end of batch 6; log content: {'loss':

End epoch 174 of training; log content: {'loss': 11.291980743408203, 'mae': 2.573643684387207, 'mse': 11.291980743408203, 'val_loss': 10.011096954345703, 'val_mae': 2.445298433303833, 'val_mse': 10.011096954345703}
...Training: end of batch 0; log content: {'loss': 10.098746299743652, 'mae': 2.3413033485412598, 'mse': 10.098746299743652}
...Training: end of batch 1; log content: {'loss': 14.49561882019043, 'mae': 2.7893435955047607, 'mse': 14.49561882019043}
...Training: end of batch 2; log content: {'loss': 12.478515625, 'mae': 2.554910898208618, 'mse': 12.478515625}
...Training: end of batch 3; log content: {'loss': 13.329014778137207, 'mae': 2.743533134460449, 'mse': 13.329014778137207}
...Training: end of batch 4; log content: {'loss': 12.412055969238281, 'mae': 2.625807523727417, 'mse': 12.412055969238281}
...Training: end of batch 5; log content: {'loss': 11.858630180358887, 'mae': 2.5915772914886475, 'mse': 11.858630180358887}
...Training: end of batch 6; log content: {'loss': 1

...Training: end of batch 3; log content: {'loss': 10.248334884643555, 'mae': 2.587010145187378, 'mse': 10.248334884643555}
...Training: end of batch 4; log content: {'loss': 10.638434410095215, 'mae': 2.606750965118408, 'mse': 10.638434410095215}
...Training: end of batch 5; log content: {'loss': 10.939873695373535, 'mae': 2.610248327255249, 'mse': 10.939873695373535}
...Training: end of batch 6; log content: {'loss': 11.34882926940918, 'mae': 2.6258223056793213, 'mse': 11.34882926940918}
...Training: end of batch 7; log content: {'loss': 11.323558807373047, 'mae': 2.5732691287994385, 'mse': 11.323558807373047}
End epoch 185 of training; log content: {'loss': 11.323558807373047, 'mae': 2.5732691287994385, 'mse': 11.323558807373047, 'val_loss': 10.019936561584473, 'val_mae': 2.440904378890991, 'val_mse': 10.019936561584473}
...Training: end of batch 0; log content: {'loss': 5.464108467102051, 'mae': 1.8658342361450195, 'mse': 5.464108467102051}
...Training: end of batch 1; log content:

...Training: end of batch 4; log content: {'loss': 11.849218368530273, 'mae': 2.6018729209899902, 'mse': 11.849218368530273}
...Training: end of batch 5; log content: {'loss': 11.50069808959961, 'mae': 2.5721538066864014, 'mse': 11.50069808959961}
...Training: end of batch 6; log content: {'loss': 11.498367309570312, 'mae': 2.5616977214813232, 'mse': 11.498367309570312}
...Training: end of batch 7; log content: {'loss': 11.356362342834473, 'mae': 2.5678436756134033, 'mse': 11.356362342834473}
End epoch 195 of training; log content: {'loss': 11.356362342834473, 'mae': 2.5678436756134033, 'mse': 11.356362342834473, 'val_loss': 10.046046257019043, 'val_mae': 2.4511568546295166, 'val_mse': 10.046046257019043}
...Training: end of batch 0; log content: {'loss': 8.762008666992188, 'mae': 2.4012558460235596, 'mse': 8.762008666992188}
...Training: end of batch 1; log content: {'loss': 11.206226348876953, 'mae': 2.597317695617676, 'mse': 11.206226348876953}
...Training: end of batch 2; log conte

# Custom Callback N. 2

Let us write another example of custom callback. This time, let's implement a custom early stopping mechanism on a pre-defined Validation MAE value.

In [16]:
class MyEarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if(logs['val_mae']< 10.0):
            print("\nReached MAE < 10.0, so cancelling training!")
            self.model.stop_training = True


In [17]:
my_es_callback = MyEarlyStopping()

model = get_model()
history = model.fit(train_features, train_labels, epochs=200, validation_split = 0.2, callbacks=[my_es_callback])

Epoch 1/200
8/8 [==============================] - 0s 18ms/step - loss: 587.5167 - mae: 23.0101 - mse: 587.5167 - val_loss: 556.3007 - val_mae: 22.8800 - val_mse: 556.3007
Epoch 2/200
8/8 [==============================] - 0s 3ms/step - loss: 515.3938 - mae: 22.1182 - mse: 515.3938 - val_loss: 502.1757 - val_mae: 22.0296 - val_mse: 502.1757
Epoch 3/200
8/8 [==============================] - 0s 3ms/step - loss: 473.7619 - mae: 21.3449 - mse: 473.7619 - val_loss: 466.4300 - val_mae: 21.2633 - val_mse: 466.4300
Epoch 4/200
8/8 [==============================] - 0s 3ms/step - loss: 438.5339 - mae: 20.5317 - mse: 438.5339 - val_loss: 435.8839 - val_mae: 20.5349 - val_mse: 435.8839
Epoch 5/200
8/8 [==============================] - 0s 3ms/step - loss: 406.1021 - mae: 19.7481 - mse: 406.1021 - val_loss: 407.7557 - val_mae: 19.8639 - val_mse: 407.7557
Epoch 6/200
8/8 [==============================] - 0s 3ms/step - loss: 376.3737 - mae: 18.9950 - mse: 376.3737 - val_loss: 381.6593 - val_mae: 1